# Lab: Aprendizaje por refuerzo (I)
# 1 - Gymnasium y entorno FrozenLake-v2

En esta sesión vamos a ver Gymnasium, una librería de referencia para probar algoritmos de aprendizaje por refuerzo que incluye algunos entornos y además define una interfaz común para integrar entornos nuevos basada en el bucle del agente inteligente que ya hemos visto en clase de teoría.

## Configuración y dependencias

In [2]:
# Declaración de constantes
SLIPPERY = False
NUM_EPISODES = 50

In [3]:
!pip install gymnasium seaborn numpy pygame

  Using cached cloudpickle-3.1.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached Farama_Notifications-0.0.4-py3-none-any.whl.metadata (558 bytes)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached pytz-2025.1-py2.py3-none-any.whl.metadata (22 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 965.4/965.4 kB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.1/16.1 MB 58.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 17.0 MB/s eta 0:00:0000:0100:01
Using cached cloudpickle-3.1.1-py3-none-any.whl (20 kB)
Using cached Farama_Notifications-0.0.4-py3-none-any.whl (2.5 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 56.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 58.1 MB/s eta 0:00:0000:01
Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 55.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 

In [4]:
import gymnasium as gym
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

## Entorno FrozenLake-v1

Vamos a hacer pruebas con el entorno FrozenLake-v1. En el enlace [https://gymnasium.farama.org/environments/toy_text/frozen_lake/](https://gymnasium.farama.org/environments/toy_text/frozen_lake/) tenéis una descripción detallada del entorno. Dedicadle unos minutos a leerlo con calma para poder entender el efecto de los diferentes algoritmos sobre el proceso de aprendizaje.

Lo primero que vamos a hacer es ejecutar el entorno con una política totalmente aleatoria. Cuando ejecutéis la siguiente celda aparecerá una ventana con el entorno. Puede que se oculte detrás de las demás ventanas por lo que quizá tendréis que buscarla. 

In [6]:
import gymnasium as gym
env = gym.make("FrozenLake-v1", desc=None, map_name="4x4", render_mode="human", is_slippery=False) 
env.reset()
env.render()
is_done = False
t = 0

while not is_done:
   action = env.action_space.sample()
   state, reward, is_done, truncated, _ = env.step(action)
   env.render()
   t +=1
        
print("\nlast state  =", state)
print("reward =     ", reward)
print("terminated = ", is_done)
print("truncated  = ", truncated)
print("time steps = ", t)


last state  = 12
reward =      0.0
terminated =  True
truncated  =  False
time steps =  23


Como vemos, con la función `step()` del entorno ejecutamos una acción. En este caso, con la llamada a `env.action_space.sample()` estamos obteniendo una acción aleatoria del espacio de acciones (el elemento `A` de la tupla del MDP). Como resultado de ejecutar la acción, recibimos:

* `state`: el nuevo estado en el que está el agente.
* `reward`: la recompensa obtenida al haber llegado al nuevo estado desde el anterior mediante la acción ejecutada.
* `terminated`: un booleano que representa si hemos llegado a un estado final.
* `truncated`: un booleano que nos dice si el entorno se ha detenido de manera inesperada (e.g. por un hipotético límite de tiempo).
* `info`: una estructura con metadatos y monitorización, que no usaremos en esta sesión.

Vamos ahora a encapsular esta política en una clase que representará nuestro agente y que iremos evolucionando y modificando:

In [7]:
class RandomAgent:
    def __init__(self, env):
        self.env = env

    def select_action(self):
        return self.env.action_space.sample()

Vamos a definir también el entorno que vamos a usar en el resto de ejemplos:

In [ ]:
env = gym.make("FrozenLake-v1", desc=None, map_name="4x4", render_mode="human", is_slippery=SLIPPERY)

Replicamos ahora el resultado de antes, pero ahora con la clase RandomAgent:

In [ ]:
env.reset()
env.render()
is_done = False
t = 0
agent = RandomAgent(env)
while not is_done:
   action = agent.select_action()
   state, reward, is_done, truncated, _ = env.step(action)
   env.render()
   t += 1
        
print("\nlast state  =", state)
print("reward =     ", reward)
print("terminated = ", is_done)
print("truncated  = ", truncated)
print("time steps = ", t)

Obviamente parece que el agente no va a resolver esta tarea (a menos que hayáis tenido muchísima suerte). Vamos a realizar unas cuantas ejecuciones para sacar resultados empíricos:

In [ ]:
def test_episode(agent, env):
    env.reset()
    is_done = False

    while not is_done:
        action = agent.select_action()
        state, reward, is_done, truncated, info = env.step(action)
    return state, reward, is_done, truncated, info

agent = RandomAgent(env)

solved = 0
rewards = []
for episode in range(50):
    state, reward, is_done, truncated, _ = test_episode(agent, env)
    rewards.append(reward)
    if state == 15:
        solved += 1

print("Solved", solved, "times (", solved * 2, "%)")  

In [ ]:
data = pd.DataFrame({'Episode': range(1, len(rewards) + 1), 'Reward': rewards})
plt.figure(figsize=(10, 6))
sns.lineplot(x='Episode', y='Reward', data=data)

plt.title('Rewards Over Episodes')
plt.xlabel('Episode')
plt.ylabel('Reward')
plt.grid(True)
plt.tight_layout()

plt.show()

En el siguiente notebook vamos a comenzar a aplicar algoritmos más fiables.